In [ ]:
def country_parser(ans):
    parsing_dict = {'Россия': 'russia', 'Великобритания': 'united-kingdom', 'США': 'united-states', 'Германия': 'germany',
                    'Франция': 'france', 'Япония': 'japan'}
    for country in parsing_dict:
        if country == ans.text:
            url = f'https://ru.investing.com/equities/{parsing_dict[country]}'
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    tree = response.content
    df = pd.read_html(tree, thousands='.', encoding=None, decimal=',')
    df = df[0]
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.drop(['Unnamed: 9'], axis=1)
    columns_names = ['Название компании', 'Текущая цена', 'Максимальная цена за сегодня',
                     'Минимальная цена за сегодня',
                     'Изменение в базисных пунктах', 'Изменение в процентах', 'Объем торгов', 'Время']
    df.columns = columns_names
    df.to_csv('Shares.csv', 'w', index=False, index_label=False)
    # Ссылки на каждую акцию
    links_tree = BeautifulSoup(tree, features="lxml")
    links_tree = links_tree.find_all('td', {'class': 'bold left noWrap elp plusIconTd'})
    d = defaultdict()
    for link in range(1, len(links_tree) + 1):
        d[link] = 'https://ru.investing.com' + links_tree[link - 1].a.get('href')
    with open('links.json', 'w') as json_file:
        json.dump(d, json_file)
        
#Функция для вывода динамики по акции
def dynamics_func(current_share):
    with open('links.json') as json_file:
        dictionary = json.load(json_file)
    #Парсинг таблицы
    try:
        url = dictionary[f'{current_share.text}']
        response = requests.get(url)
        tree = BeautifulSoup(response.content, features="lxml")
        data = tree.find('div', {'class': 'instrument-page_section__79xMl'})
        data = tree.find('ul', {'class': 'list_list--horizontal__3fUDd navbar_navbar-sub__1_y2w'})
        data_list = []
        for j in data:
            data_list.append(j)
        data = 'https://ru.investing.com' + data_list[2].a.get('href')
        response = requests.get(data, headers={'User-Agent': UserAgent().chrome})
        tree = response.content
        df = pd.read_html(tree, attrs={'id': 'curr_table'}, decimal=',', thousands='.')
        df = df[0]
        # Преобразование даты
        dates = []
        for ind in range(len(df['Дата'])):
            if len(df['Дата'][ind].astype(str)[:-4]) == 4:
                date = df['Дата'][ind].astype(str)[:-4][:2]
                month = df['Дата'][ind].astype(str)[:-4][-2:]
                final_date = date + '.' + month
                dates.append(final_date)
            elif len(df['Дата'][ind].astype(str)[:-4]) == 3:
                date = '0' + df['Дата'][ind].astype(str)[:-4][:1]
                month = df['Дата'][ind].astype(str)[:-4][-2:]
                final_date = date + '.' + month
                dates.append(final_date)
        df['Дата'] = dates